# Adição de Clientes, Contas e Chaves PIX com Retry e Consistência

Este notebook demonstra como adicionar novos clientes, contas e chaves PIX ao banco MKL, garantindo IDs consistentes, ingestão em lote e em tempo real, e lógica de retry para operações que podem falhar.

---

## 1. Importação de Bibliotecas Necessárias
Importe as bibliotecas essenciais para manipulação de dados, conexão com banco, simulação de falhas e delays.

In [ ]:
# Importação de bibliotecas
import time
import random
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
from tqdm import tqdm
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Configurações dinâmicas do banco de dados
DB_CONFIG = {
    'host': os.getenv('POSTGRES_HOST', 'localhost'),
    'port': os.getenv('POSTGRES_PORT', '5432'),
    'database': os.getenv('POSTGRES_DB', 'mkl_bank'),
    'schema': os.getenv('POSTGRES_SCHEMA', 'mkl_bank'),
    'user': os.getenv('POSTGRES_USER', 'postgres'),
    'password': os.getenv('POSTGRES_PASSWORD', 'password')
}

# Configurações dinâmicas das tabelas
TABLE_CONFIG = {
    'clientes_table': os.getenv('CLIENTES_TABLE', 'clientes'),
    'contas_table': os.getenv('CONTAS_TABLE', 'contas'),
    'chaves_pix_table': os.getenv('CHAVES_PIX_TABLE', 'chaves_pix'),
    'cartoes_table': os.getenv('CARTOES_TABLE', 'cartoes'),
    'transacoes_table': os.getenv('TRANSACOES_TABLE', 'transacoes')
}

print("✅ Configurações carregadas do arquivo .env:")
print(f"   📊 Schema: {DB_CONFIG['schema']}")
print(f"   🏦 Tabela Clientes: {TABLE_CONFIG['clientes_table']}")
print(f"   💳 Tabela Contas: {TABLE_CONFIG['contas_table']}")
print(f"   📱 Tabela Chaves PIX: {TABLE_CONFIG['chaves_pix_table']}")

# ...adicione outras bibliotecas conforme necessário...

## 2. Função de Retry para Operações

Defina uma função que tenta executar uma operação e faz novas tentativas em caso de falha, com delays progressivos.

In [ ]:
def retry_operation(operation, max_retries=3, delay=2, backoff=2):
    """
    Executa uma operação com lógica de retry progressivo.
    operation: função a ser executada
    max_retries: número máximo de tentativas
    delay: tempo inicial de espera
    backoff: fator de multiplicação do delay
    """
    attempt = 0
    while attempt < max_retries:
        try:
            return operation()
        except Exception as e:
            print(f"⚠️ Falha na tentativa {attempt+1}: {e}")
            time.sleep(delay)
            delay *= backoff
            attempt += 1
    raise Exception(f"❌ Operação falhou após {max_retries} tentativas.")

## 3. Configuração Dinâmica e Inserção de Dados

### 📋 Configuração do Arquivo .env

Todas as queries e configurações de banco são dinâmicas e baseadas no arquivo `.env`. Exemplo de configuração:

```env
# Configurações do Banco de Dados
POSTGRES_HOST=sua-instancia-azure.postgres.database.azure.com
POSTGRES_PORT=5432
POSTGRES_DB=mkl_bank
POSTGRES_SCHEMA=mkl_bank
POSTGRES_USER=seu_usuario
POSTGRES_PASSWORD=sua_senha

# Configurações de Tabelas (permite customização)
CLIENTES_TABLE=clientes
CONTAS_TABLE=contas
CHAVES_PIX_TABLE=chaves_pix
CARTOES_TABLE=cartoes
TRANSACOES_TABLE=transacoes

# Configurações de Processamento
BATCH_SIZE=1000
REAL_TIME_DELAY=0.1
```

### 🔧 Funcionalidades

- **Queries Dinâmicas:** Todas as consultas SQL são construídas dinamicamente usando as variáveis do `.env`
- **Configuração Flexível:** Permite alterar nomes de tabelas, schema e conexão sem modificar código
- **Inserção em Lote e Tempo Real:** Suporte para ambos os modos de inserção
- **Retry Logic:** Operações robustas com tentativas automáticas em caso de falha

In [ ]:
# Funções utilitárias para geração de dados consistentes
import uuid
from faker import Faker
faker = Faker('pt_BR')

# Função para criar conexão dinâmica
def create_connection():
    """Cria conexão com o banco usando configurações do .env"""
    try:
        connection = psycopg2.connect(
            host=DB_CONFIG['host'],
            port=DB_CONFIG['port'],
            database=DB_CONFIG['database'],
            user=DB_CONFIG['user'],
            password=DB_CONFIG['password']
        )
        print(f"✅ Conectado ao banco: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")
        return connection
    except Exception as e:
        print(f"❌ Erro na conexão: {e}")
        raise

# Carregar dados existentes para garantir consistência (QUERIES DINÂMICAS)
def get_existing_ids(connection):
    """Busca IDs existentes usando configurações dinâmicas do .env"""
    cursor = connection.cursor()
    
    # Queries dinâmicas baseadas no .env
    schema = DB_CONFIG['schema']
    clientes_table = TABLE_CONFIG['clientes_table']
    contas_table = TABLE_CONFIG['contas_table'] 
    chaves_pix_table = TABLE_CONFIG['chaves_pix_table']
    
    # Query dinâmica para clientes
    query_clientes = f"SELECT id_cliente FROM {schema}.{clientes_table}"
    print(f"🔍 Executando: {query_clientes}")
    cursor.execute(query_clientes)
    clientes = [row[0] for row in cursor.fetchall()]
    
    # Query dinâmica para contas
    query_contas = f"SELECT id_conta FROM {schema}.{contas_table}"
    print(f"🔍 Executando: {query_contas}")
    cursor.execute(query_contas)
    contas = [row[0] for row in cursor.fetchall()]
    
    # Query dinâmica para chaves PIX
    query_chaves_pix = f"SELECT chave_pix FROM {schema}.{chaves_pix_table}"
    print(f"🔍 Executando: {query_chaves_pix}")
    cursor.execute(query_chaves_pix)
    chaves_pix = [row[0] for row in cursor.fetchall()]
    
    cursor.close()
    
    print(f"📊 Dados existentes encontrados:")
    print(f"   👥 Clientes: {len(clientes)}")
    print(f"   🏦 Contas: {len(contas)}")
    print(f"   📱 Chaves PIX: {len(chaves_pix)}")
    
    return clientes, contas, chaves_pix

# Gerar novos clientes
def generate_new_clients(n):
    """Gera novos clientes com dados realistas"""
    clients = []
    for _ in range(n):
        nome = faker.name()
        cpf = faker.cpf()
        email = faker.email()
        id_cliente = str(uuid.uuid4())
        clients.append({
            'id_cliente': id_cliente,
            'nome_cliente': nome,
            'cpf': cpf,
            'email': email
        })
    print(f"✅ Gerados {len(clients)} novos clientes")
    return clients

# Gerar novas contas para clientes
def generate_new_accounts(clients):
    """Gera novas contas para os clientes fornecidos"""
    accounts = []
    for client in clients:
        id_conta = str(uuid.uuid4())
        numero_conta = faker.random_number(digits=8)
        tipo_conta = random.choice(['corrente', 'poupanca'])
        accounts.append({
            'id_conta': id_conta,
            'id_cliente': client['id_cliente'],
            'numero_conta': numero_conta,
            'tipo_conta': tipo_conta
        })
    print(f"✅ Geradas {len(accounts)} novas contas")
    return accounts

# Gerar novas chaves PIX para contas
def generate_new_pix_keys(accounts):
    """Gera novas chaves PIX para as contas fornecidas"""
    pix_keys = []
    for account in accounts:
        chave_pix = faker.uuid4()
        tipo_chave = random.choice(['cpf', 'email', 'telefone', 'aleatoria'])
        pix_keys.append({
            'chave_pix': chave_pix,
            'id_conta': account['id_conta'],
            'tipo_chave': tipo_chave
        })
    print(f"✅ Geradas {len(pix_keys)} novas chaves PIX")
    return pix_keys

# Inserção em lote com retry (QUERY DINÂMICA)
def insert_batch(connection, table_key, data, columns):
    """Insere dados em lote usando configurações dinâmicas"""
    def operation():
        cursor = connection.cursor()
        schema = DB_CONFIG['schema']
        table_name = TABLE_CONFIG[table_key]
        
        # Query dinâmica
        query = f"INSERT INTO {schema}.{table_name} ({', '.join(columns)}) VALUES %s"
        print(f"📝 Query de inserção: {query}")
        
        values = [[row[col] for col in columns] for row in data]
        execute_batch(cursor, query.replace('%s', '(' + ','.join(['%s']*len(columns)) + ')'), values)
        connection.commit()
        cursor.close()
        print(f"✅ Inseridos {len(data)} registros em {schema}.{table_name}")
    
    retry_operation(operation)

# Inserção em tempo real (um a um) (QUERY DINÂMICA)
def insert_realtime(connection, table_key, data, columns):
    """Insere dados em tempo real usando configurações dinâmicas"""
    schema = DB_CONFIG['schema']
    table_name = TABLE_CONFIG[table_key]
    
    print(f"🔄 Iniciando inserção em tempo real em {schema}.{table_name}")
    
    for row in tqdm(data, desc=f'Inserindo em {table_name}'):
        def operation():
            cursor = connection.cursor()
            query = f"INSERT INTO {schema}.{table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s']*len(columns))})"
            cursor.execute(query, [row[col] for col in columns])
            connection.commit()
            cursor.close()
        retry_operation(operation)
    
    print(f"✅ Inserção em tempo real concluída: {len(data)} registros")

# Exemplo de uso com configurações dinâmicas:
# connection = create_connection()
# clientes = generate_new_clients(10)
# contas = generate_new_accounts(clientes)
# pix_keys = generate_new_pix_keys(contas)
# insert_batch(connection, 'clientes_table', clientes, ['id_cliente','nome_cliente','cpf','email'])
# insert_batch(connection, 'contas_table', contas, ['id_conta','id_cliente','numero_conta','tipo_conta'])
# insert_batch(connection, 'chaves_pix_table', pix_keys, ['chave_pix','id_conta','tipo_chave'])

## 4. Teste da Função de Retry com Operação que Falha

Simule uma operação que falha aleatoriamente e utilize a função de retry para demonstrar sua eficácia.

## 📋 Exemplo Prático: Inserção com Configurações Dinâmicas

Demonstração completa de como usar as funções com configurações do arquivo `.env`:

In [ ]:
# Exemplo prático de inserção com configurações dinâmicas
def exemplo_insercao_completa(quantidade_clientes=5, modo_insercao='lote'):
    """
    Exemplo completo de inserção de clientes, contas e chaves PIX
    
    Args:
        quantidade_clientes (int): Número de clientes a serem criados
        modo_insercao (str): 'lote' ou 'tempo_real'
    """
    print(f"🚀 INICIANDO INSERÇÃO DE {quantidade_clientes} CLIENTES")
    print(f"📊 Modo: {modo_insercao.upper()}")
    print("=" * 50)
    
    try:
        # 1. Conectar ao banco com configurações dinâmicas
        print("🔌 Conectando ao banco...")
        connection = create_connection()
        
        # 2. Verificar dados existentes (usando queries dinâmicas)
        print("\n🔍 Verificando dados existentes...")
        clientes_existentes, contas_existentes, pix_existentes = get_existing_ids(connection)
        
        # 3. Gerar novos dados
        print(f"\n📝 Gerando {quantidade_clientes} novos registros...")
        novos_clientes = generate_new_clients(quantidade_clientes)
        novas_contas = generate_new_accounts(novos_clientes)
        novas_chaves_pix = generate_new_pix_keys(novas_contas)
        
        # 4. Inserir dados com base no modo escolhido
        print(f"\n💾 Inserindo dados em modo {modo_insercao}...")
        
        if modo_insercao.lower() == 'lote':
            # Inserção em lote
            print("📦 Inserindo em lote...")
            insert_batch(connection, 'clientes_table', novos_clientes, 
                        ['id_cliente', 'nome_cliente', 'cpf', 'email'])
            insert_batch(connection, 'contas_table', novas_contas, 
                        ['id_conta', 'id_cliente', 'numero_conta', 'tipo_conta'])
            insert_batch(connection, 'chaves_pix_table', novas_chaves_pix, 
                        ['chave_pix', 'id_conta', 'tipo_chave'])
        
        elif modo_insercao.lower() == 'tempo_real':
            # Inserção em tempo real
            print("⏱️ Inserindo em tempo real...")
            insert_realtime(connection, 'clientes_table', novos_clientes, 
                           ['id_cliente', 'nome_cliente', 'cpf', 'email'])
            insert_realtime(connection, 'contas_table', novas_contas, 
                           ['id_conta', 'id_cliente', 'numero_conta', 'tipo_conta'])
            insert_realtime(connection, 'chaves_pix_table', novas_chaves_pix, 
                           ['chave_pix', 'id_conta', 'tipo_chave'])
        
        # 5. Verificar resultados
        print("\n✅ Verificando resultados...")
        novos_clientes_bd, novas_contas_bd, novas_pix_bd = get_existing_ids(connection)
        
        print(f"\n🎉 INSERÇÃO CONCLUÍDA COM SUCESSO!")
        print(f"   📈 Clientes: {len(clientes_existentes)} → {len(novos_clientes_bd)} (+{len(novos_clientes_bd) - len(clientes_existentes)})")
        print(f"   📈 Contas: {len(contas_existentes)} → {len(novas_contas_bd)} (+{len(novas_contas_bd) - len(contas_existentes)})")
        print(f"   📈 Chaves PIX: {len(pix_existentes)} → {len(novas_pix_bd)} (+{len(novas_pix_bd) - len(pix_existentes)})")
        
        connection.close()
        
    except Exception as e:
        print(f"❌ ERRO durante a inserção: {e}")
        if 'connection' in locals():
            connection.close()
        raise

# Demonstração com diferentes configurações
print("💡 EXEMPLOS DE USO:")
print("   exemplo_insercao_completa(10, 'lote')      # 10 clientes em lote")
print("   exemplo_insercao_completa(5, 'tempo_real') # 5 clientes em tempo real")
print("\n🔧 Para executar, descomente e execute:")
print("# exemplo_insercao_completa(5, 'lote')")

# Descomente para executar:
# exemplo_insercao_completa(5, 'lote')

In [ ]:
# Simulação de operação que falha aleatoriamente
def random_fail_operation():
    if random.random() < 0.7:
        raise Exception("Falha simulada!")
    return "Sucesso!"

# Testando a função de retry

try:
    result = retry_operation(random_fail_operation, max_retries=5, delay=1)
    print(f"Resultado da operação: {result}")
except Exception as e:
    print(str(e))

## 🎯 Personalização e Configurações Avançadas

### 📝 Como Personalizar Queries via .env

1. **Nomes de Tabelas:**
   ```env
   CLIENTES_TABLE=meus_clientes
   CONTAS_TABLE=minhas_contas
   CHAVES_PIX_TABLE=chaves_pix_personalizadas
   ```

2. **Schema Diferente:**
   ```env
   POSTGRES_SCHEMA=banco_producao
   ```

3. **Configurações de Processamento:**
   ```env
   BATCH_SIZE=500
   REAL_TIME_DELAY=0.05
   MAX_RETRIES=5
   ```

### 🔧 Funcionalidades Disponíveis

| Função | Descrição | Exemplo |
|--------|-----------|---------|
| `create_connection()` | Conexão dinâmica | Usa configurações do .env |
| `get_existing_ids()` | Queries dinâmicas | Schema e tabelas configuráveis |
| `generate_new_clients(n)` | Gera clientes | Dados realistas com Faker |
| `insert_batch()` | Inserção em lote | Alta performance |
| `insert_realtime()` | Inserção tempo real | Com progresso visual |
| `retry_operation()` | Operações robustas | Retry automático em falhas |

### ✅ Vantagens da Configuração Dinâmica

- **🔄 Flexibilidade:** Mudanças sem alterar código
- **🛡️ Segurança:** Credenciais no .env (não no código)
- **🎯 Ambientes:** Dev, teste, produção facilmente
- **📊 Monitoramento:** Logs detalhados de todas operações
- **⚡ Performance:** Inserção otimizada em lote ou tempo real

---
**🎉 Sistema pronto para uso em qualquer ambiente!**